In [1]:
import sys
sys.path.append('../')
import networkx as nx
import time
from smodels.theory.exceptions import SModelSTheoryError as SModelSError
from smodels.tools.smodelsLogging import logger
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
from smodels.theory.element import Element
from smodels.theory.topology import TopologyDict
from smodels.theory.crossSection import XSection,XSectionInfo,XSectionList
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.tools.physicsUnits import fb, GeV
from smodels.theory.tree import Tree,ParticleNode
from smodels.theory.decomposer import cascadeDecay, addOneStepDecays
import itertools
%load_ext line_profiler

### Decomposer

In [2]:
def decompose(model, sigmacut= 0*fb, doCompress=True, doInvisible=True,
              minmassgap= 0*GeV):
    """
    Perform decomposition using the information stored in model.
    
    :param sigmacut: minimum sigma*BR to be generated, by default sigmacut = 0.1 fb
    :param doCompress: turn mass compression on/off
    :param doInvisible: turn invisible compression on/off
    :param minmassgap: maximum value (in GeV) for considering two R-odd particles
                       degenerate (only revelant for doCompress=True )
    :returns: list of topologies (TopologyList object)

    """
    t1 = time.time()
    
    xSectionList = model.xsections    
    pdgList = model.getValuesFor('pdg')

    if doCompress and minmassgap/GeV < 0.:
        logger.error("Asked for compression without specifying minmassgap. Please set minmassgap.")        
        raise SModelSError()

    if isinstance(sigmacut,(float,int)):
        sigmacut = float(sigmacut) * fb

    xSectionList.removeLowerOrder()
    # Order xsections by highest xsec value to improve performance
    xSectionList.sort()

    # Generate all primary nodes (e.g. PV > X+Y)
    # and assign the nodeWeight as the maximum cross-section
    productionTrees = []
    for pid in xSectionList.getPIDpairs():
        weight = xSectionList.getXsecsFor(pid)
        if weight < sigmacut:
            continue
        pv = ParticleNode(model.getParticlesWith(label='PV')[0],0,nodeWeight=weight)
        pv.xsection = xSectionList.getXsecsFor(pid)
        primaryMothers = [ParticleNode(model.getParticlesWith(pdg=pdg)[0],i+1) for i,pdg in enumerate(pid)]
        productionTrees.append(Tree({pv : primaryMothers}))

    # Sort production trees
    productionTrees = sorted(productionTrees, key = lambda t: t.getTreeWeight().getMaxXsec(), reverse=True)
    
    
#     print('%i production trees' %len(productionTrees))

    # For each production tree, produce all allowed cascade decays (above sigmacut):
    allTrees = []
    for tree in productionTrees:
        allTrees += cascadeDecay(tree,sigmacut=sigmacut)

#     print('%i decayed trees' %len(allTrees))

    # Create elements for each tree and combine equal elements
    smsTopDict = TopologyDict()

    for tree in allTrees:
        newElement = Element(tree)
        newElement.weight = tree.getTreeWeight()
        smsTopDict.addElement(newElement)                                                 

    if doCompress or doInvisible:
        smsTopDict.compressElements(doCompress, doInvisible, minmassgap)
    smsTopDict._setElementIds()    
#     print('total number of unique elements = %i' %len(smsTopDict.getElements()))
#     print("decomposer done in %.2f s." % (time.time() -t1 ) )
    

    return smsTopDict

### Load model

In [3]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
# slhafile = '../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Check with large sigmacut

In [4]:
sigmacut = 10*fb
t0 = time.time()
topDict = decompose(model, sigmacut= sigmacut, doCompress=False, doInvisible=False)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
nUnique = 103 # uncompressed
print('%i unique elements' %(len(topDict.getElements())))
print('%i unique elements (expected)' %nUnique)

decomposer done in 0.82 s.
103 unique elements
103 unique elements (expected)


In [5]:
for c in sorted(topDict.keys()):
    total = 0.0*fb
    for el in topDict[c]:
        total += el.weight.getMaxXsec()
    print(c,len(topDict[c]),total)

110110101000 2 1.08E-01 [pb]
111010100110101000 58 6.13E+00 [pb]
11101010011011010000 1 3.19E-02 [pb]
111010100110101101010000 7 4.91E-01 [pb]
11101010011011011010100000 4 8.54E-02 [pb]
11101101000110101101010000 4 8.11E-02 [pb]
111010110101000110101101010000 27 1.24E+00 [pb]


### Check with small sigmacut

In [6]:
sigmacut = 0.1*fb
t0 = time.time()
topDict = decompose(model, sigmacut= sigmacut, doCompress=False, doInvisible=False)
print("decomposer done in %.2f s." % (time.time() -t0 ) )
print('%i unique elements' %(len(topDict.getElements())))

decomposer done in 54.74 s.
5435 unique elements


In [12]:
topSummary = []
for c in sorted(topDict.keys()):
    total = 0.0*fb
    for el in topDict[c]:
        total += el.weight.getMaxXsec()
    print(c,len(topDict[c]),total)
    topSummary.append([len(topDict[c]),round(total.asNumber(fb),3)])
expectedSummary = [[1, 3.883], [2, 1.003], [3, 1.094], [4, 0.892], [4, 2.616], [6, 2.853], [8, 3.643], [10, 3.376], [10, 14.988], [11, 10.526], [13, 13.994], [14, 11.171], [15, 164.364], [16, 78.045], [17, 30.516], [25, 12.91], [28, 19.106], [49, 42.873], [56, 25.215], [62, 48.534], [78, 51.703], [87, 150.926], [90, 6350.441], [102, 154.363], [104, 94.307], [104, 153.527], [107, 404.754], [113, 403.824], [126, 47.333], [132, 1078.776], [144, 267.975], [183, 106.275], [295, 166.074], [304, 377.261], [411, 157.421], [504, 2817.347], [648, 455.245], [747, 2086.069], [802, 1330.765]]


print('Equal?',sorted(topSummary) == sorted(expectedSummary))

110100 1 3.88E-03 [pb]
1101101000 4 2.62E-03 [pb]
110110101000 15 1.64E-01 [pb]
11101001101000 10 3.38E-03 [pb]
1101101101010000 25 1.29E-02 [pb]
1110100110101000 49 4.29E-02 [pb]
111010011011010000 2 1.00E-03 [pb]
111010100110101000 90 6.35E+00 [pb]
11101001101101010000 126 4.73E-02 [pb]
11101010011011010000 16 7.80E-02 [pb]
1110100110101101010000 13 1.40E-02 [pb]
1110101001101011010000 17 3.05E-02 [pb]
1110101001101101010000 295 1.66E-01 [pb]
1110110100011011010000 10 1.50E-02 [pb]
111010011011011010100000 8 3.64E-03 [pb]
111010100110101101010000 132 1.08E+00 [pb]
111011010001101011010000 14 1.12E-02 [pb]
111011010001101101010000 11 1.05E-02 [pb]
11101001101011011010100000 4 8.92E-04 [pb]
11101010011011011010100000 113 4.04E-01 [pb]
11101011010001101011010000 3 1.09E-03 [pb]
11101011010001101101010000 6 2.85E-03 [pb]
11101101000110101101010000 107 4.05E-01 [pb]
11101101010001101101010000 411 1.57E-01 [pb]
1110101001101011011010100000 144 2.68E-01 [pb]
1110101101000110101101010000 102

### Check with mas compression

In [8]:
# %lprun -f addOneStepDecays decompose(model, sigmacut= sigmacut)